<h1 style="color:DodgerBlue">Индивидуальный проект</h1>

<h2 style="color:DodgerBlue">Название проекта: Invoice -- Счет-фактура</h2> 

----

### Вариант задания: № 10


<h2 style="color:DodgerBlue">Описание проекта:</h2>

----

Создать базовый класс Invoice в С#, который будет представлять информацию о фактурах за поставленные товары или оказанные услуги.

На основе этого класса разработать 2-3 производных класса, демонстрирующих принципы наследования и полиморфизма. 

В каждом из классов должны быть реализованы новые атрибуты и методы, а также переопределены некоторые методы базового класса для демонстрации полиморфизма.

Требования к базовому классу Invoice:

Атрибуты  : 
* Номер фактуры (InvoiceNumber), 
* Дата выдачи (IssueDate), 
* Общая сумма (TotalAmount).

Методы:

* Calculate Total(): метод для расчета общей суммы по фактуре.
        
* AddLine(Lineltem lineltem): метод для добавления позиции в фактуру.

* RemoveLine(Lineltem lineltem): метод для удаления позиции из фактуры.

Требования к производным классам:

* ТоварнаяФактура (GoodsInvoice): Должна содержать дополнительные атрибуты, такие как Дата поставки (SupplyDate). 

* Метод AddLine() должен быть переопределен для добавления информации о дате поставки товара при добавлении позиции.

* УслуговаяФактура (Servicelnvoice): Должна содержать дополнительные атрибуты, такие как Дата оказания услуги (ServiceDate). Метод RemoveLine() должен быть переопределен для добавления информации о причине аннулирования услуги при удалении позиции.

* КомбинированнаяФактура (CombinedInvoice) (если требуется третий класс): Должна содержать дополнительные атрибуты, такие как Наличие возврата (ReturnAllowed). Метод Calculate Total() должен быть переопределен для учета возможного возврата товара или услуги при расчете общей суммы.


<h2 style="color:DodgerBlue">Реализация:</h2>

----

In [2]:
public class Invoice
{
    public string InvoiceNumber { get; set; } //Номер фактуры
    public DateTime IssueDate { get; set; } // Дата выдачи
    public decimal TotalAmount { get; protected set; } // Общая сумма
    protected List<LineItem> LineItems { get; set; } // Список товаров на фактуре

    public Invoice(string invoiceNumber, DateTime issueDate) // инициализация
    {
        InvoiceNumber = invoiceNumber;
        IssueDate = issueDate;
        LineItems = new List<LineItem>();
    }

    public virtual void CalculateTotal() // метод для подсчета суммы товаров на фактуре
    {
        TotalAmount = 0;
        foreach (var item in LineItems)
        {
            TotalAmount += item.Amount;
        }
    }

    public virtual void AddLine(LineItem lineItem) // метод добавления товаров на фактуру, а затем пересчитывает сумму
    {
        LineItems.Add(lineItem);
        CalculateTotal();
    }

    public virtual void RemoveLine(string description) // Удаление товара с фактуры, с перерасчетом
    {
        var lineItem = LineItems.FirstOrDefault(item => item.Description == description);
        if (lineItem != null)
        {
            LineItems.Remove(lineItem);
            CalculateTotal();
        }
    }
}

public class LineItem
{
    public string Description { get; set; }// Описание позиции
    public decimal Amount { get; set; }// Стоимость

    public LineItem(string description, decimal amount) 
    {
        Description = description;
        Amount = amount;
    }
}

public class GoodsInvoice : Invoice
{
    public DateTime SupplyDate { get; set;}// дата поставки
    // base - вызов конструктора родителя для производного класса, для инициализации необходимых полей и тд.
    public GoodsInvoice(string invoiceNumber, DateTime issueDate, DateTime supplyDate): base(invoiceNumber, issueDate) 
    {
        SupplyDate = supplyDate;
    }

    public override void AddLine(LineItem lineItem) // добавляет элемент с указанием даты доставки
    {
        base.AddLine(lineItem);
        Console.WriteLine($"Товар добавлен с датой поставки: {SupplyDate}");
    }
}

public class Servicelnvoice : Invoice
{
    public DateTime ServiceDate { get; set;} // дата оказания услуги

    public Servicelnvoice(string invoiceNumber, DateTime issueDate, DateTime serviceDate): base(invoiceNumber, issueDate)
    {
        ServiceDate = serviceDate;
    }

    public override void RemoveLine(string description) // удаляет элемент из списка, с указанием причины
    {
        base.RemoveLine(description);
        Console.WriteLine("Услуга была отменена. Пожалуйста, укажите причину.");
    }
}

public class CombinedInvoice : Invoice
{
    public bool ReturnAllowed { get; } // флаг, который определяет, разрешен ли возврат
    public decimal ReturnAmount { get; private set; } //сумма, которую нужно вычесть из полной суммы при возврате

    public CombinedInvoice(string invoiceNumber, DateTime issueDate, bool returnAllowed) : base(invoiceNumber, issueDate)
    {
        ReturnAllowed = returnAllowed;
    }

    public override void CalculateTotal() // подсчитывает общую сумму всех товарных позиций и корректирует её с учетом возврата, если возвраты разрешены
    {
        TotalAmount = 0;
        foreach (var item in LineItems)
        {
            TotalAmount += item.Amount;
        }

        // Если разрешён возврат, уменьшаем общую сумму на размер возврата
        if (ReturnAllowed)
        {
            TotalAmount -= ReturnAmount;
        }
    }

    public void ProcessReturn(string description)
    {
        base.RemoveLine(description);
        Console.WriteLine($"Товар {description} возвращен");
    }
}


GoodsInvoice myGood = new GoodsInvoice("100", DateTime.Now, DateTime.Now.AddDays(5));
myGood.AddLine(new LineItem("Товар 1", 150));
myGood.AddLine(new LineItem("Товар 2", 250));
Console.WriteLine(myGood.TotalAmount);

Servicelnvoice myService = new Servicelnvoice("101", DateTime.Now, DateTime.Now.AddDays(1));
myService.AddLine(new LineItem("Товар 1", 150));
myService.AddLine(new LineItem("Товар 2", 250));
myService.AddLine(new LineItem("Товар 3", 300));
Console.WriteLine(myService.TotalAmount);
myService.RemoveLine("Товар 2");
Console.WriteLine(myService.TotalAmount);

CombinedInvoice myCombine = new CombinedInvoice("102", DateTime.Now, true);
myCombine.AddLine(new LineItem("Товар 1", 250));
myCombine.AddLine(new LineItem("Товар 2", 270));
myCombine.AddLine(new LineItem("Товар 3", 300));
myCombine.AddLine(new LineItem("Товар 4", 350));
Console.WriteLine(myCombine.TotalAmount);
myCombine.ProcessReturn("Товар 3");
Console.WriteLine(myCombine.TotalAmount);



Товар добавлен с датой поставки: 9/19/2024 2:14:22 PM
Товар добавлен с датой поставки: 9/19/2024 2:14:22 PM
400
700
Услуга была отменена. Пожалуйста, укажите причину.
450
1170
Товар Товар 3 возвращен
870
